In [41]:
import pandas as pd
import folium # pip install folium
import seaborn as sns # pip install seaborn
import matplotlib.pyplot as plt
import plotly.express as px # pip install plotly

from collections import Counter
# from keplergl import KeplerGl

plt.rc('font', family='Malgun Gothic') # 한글 폰트 설정 (Malgun Gothic)

plt.rc('axes', unicode_minus=False) # 마이너스 기호 깨짐 방지

# 공용 변수 선언
dataIn = './../dataIn/'
dataOut = './../dataOut/'

In [42]:
# 이미 생성해놓은 매장 정보 파일을 읽어옵니다.
starbucks = pd.read_csv(dataOut + 'starbucks_file.csv')
ediya = pd.read_csv(dataOut + 'ediya_file.csv')
hollysBefore = pd.read_csv(dataOut + 'hollys_file.csv')

In [43]:
print(f'스타벅스 매장 갯수 : {len(starbucks)}')
print(f'이디야 매장 갯수 : {len(ediya)}')
print(f'할리스 매장 갯수 : {len(hollysBefore)}')

스타벅스 매장 갯수 : 666
이디야 매장 갯수 : 462
할리스 매장 갯수 : 40


In [44]:
starbucks.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [45]:
ediya.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [46]:
hollysBefore.columns

Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도', '전화 번호', '24시간', 'DT 매장',
       '주차', '테라스', '흡연시설'],
      dtype='object')

In [47]:
print('스타벅스와 할리스의 공통 컬럼 정보')
common_col = starbucks.columns.intersection(hollysBefore.columns)
common_col

스타벅스와 할리스의 공통 컬럼 정보


Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [48]:
print('다른 매장들과 공통된 컬럼들만 따로 추출')
hollys = hollysBefore[common_col]
hollys.columns

다른 매장들과 공통된 컬럼들만 따로 추출


Index(['브랜드', '상호', '주소', '시도', '군구', '위도', '경도'], dtype='object')

In [49]:
print('# 각 매장의 데이터 프레임을 하나로 모아줌')
coffeeFrame = pd.concat([starbucks,ediya, hollys],axis=0)
print(f'전체 매장 갯수 : {len(coffeeFrame)}')

# 각 매장의 데이터 프레임을 하나로 모아줌
전체 매장 갯수 : 1168


In [50]:
filename = dataOut + 'coffee_list.csv'
coffeeFrame.to_csv(filename, index=False, encoding='UTF-8')
print(f'{filename} 파일이 저장되었습니다.')

./../dataOut/coffee_list.csv 파일이 저장되었습니다.


In [51]:
coffeeFrame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1168 entries, 0 to 39
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   브랜드     1168 non-null   object 
 1   상호      1168 non-null   object 
 2   주소      1168 non-null   object 
 3   시도      1168 non-null   object 
 4   군구      1168 non-null   object 
 5   위도      1166 non-null   float64
 6   경도      1166 non-null   float64
dtypes: float64(2), object(5)
memory usage: 73.0+ KB


In [52]:
print(f'위도/경도 결측치 제거전 : {len(coffeeFrame)}')

위도/경도 결측치 제거전 : 1168


In [53]:
# notnull() 함수는 결측치가 아니면 True를 반환해주는 함수 ↔ isnull()
mapFrame = coffeeFrame[coffeeFrame['위도'].notnull()]
print(f'위도/경도 결측치 제거후 : {len(mapFrame)}')

위도/경도 결측치 제거후 : 1166


In [54]:
# 지도 위에 매장 표시
# folium에서 사용할 브랜드 별 색상지정
brand_colors = {
    '스타벅스':'beige',
    '이디야':'darkblue',
    '할리스':'lightblue'
}

seoul_center = [mapFrame['위도'].mean(), mapFrame['경도'].mean()] # 서울 중심 위치
seoul_center

[np.float64(37.54322478480824), np.float64(126.98812735399429)]

In [59]:
# 서울 지도 객체
seoul_map = folium.Map(location=seoul_center, zoom_start=11)

# iterrows() : iterable rows(전체 행을 반복)
# for 색인_번호, 데이터_1행 in mapFrame.iterrows():
# 찾아 볼것 : 마커 크기 조정, popup의 모양
for _,row in mapFrame.iterrows():
    folium.Marker(
        location=[row['위도'],row['경도']],
        popup=f"{row['상호']}({row['브랜드']})", # 팝업 예시 : xxx점(스타벅스)
        icon=folium.Icon(color=brand_colors.get(row['브랜드'],'gray'))
    ).add_to(seoul_map)
#end for

htmlfile = dataOut + 'coffee_map.html'
seoul_map.save(htmlfile)
print(f'{htmlfile} 파일 생성')

./../dataOut/coffee_map.html 파일 생성


In [58]:
seoul_map

In [62]:
print('# 군구 컬럼이 마포구, 용산구, 서대문구인 항목들만 추출')
concern_gu = ['마포구', '용산구', '서대문구']
myfilter01 = mapFrame[mapFrame['군구'].isin(concern_gu)]
print(f'갯수 : {len(myfilter01)}')

# 군구 컬럼이 마포구, 용산구, 서대문구인 항목들만 추출
갯수 : 140


In [61]:
myfilter01.head()

,브랜드,상호,주소,시도,군구,위도,경도
224,스타벅스,리저브 매장 2번,서울특별시 마포구 백범로 35 (신수동),서울특별시,마포구,37.551027,126.943134
225,스타벅스,리저브 매장 2번,서울특별시 마포구 상암산로 76 (상암동),서울특별시,마포구,37.579314,126.892210
226,스타벅스,리저브 매장 2번,서울특별시 마포구 토정로 295 (용강동),서울특별시,마포구,37.541186,126.942209
227,스타벅스,리저브 매장 2번,서울특별시 마포구 양화로 125 (서교동),서울특별시,마포구,37.554636,126.920360
228,스타벅스,리저브 매장 2번,서울특별시 마포구 마포대로 173-14 (아현동),서울특별시,마포구,37.550800,126.953432


In [63]:
my_center = [myfilter01['위도'].mean(), myfilter01['경도'].mean()] # 서울 중심 위치
my_center

[np.float64(37.55298409211924), np.float64(126.9430383047872)]

In [74]:
# 서울 지도 객체
filter_map = folium.Map(location=my_center, zoom_start=13, tiles='OpenStreetMap')

# iterrows() : iterable rows(전체 행을 반복)
# for 색인_번호, 데이터_1행 in mapFrame.iterrows():
# 찾아 볼것 : 마커 크기 조정, popup의 모양
for _,row in myfilter01.iterrows():
    # 브랜드 별로 색상을 다르게 지정하려면 brand_colors에 추가하는 방법도 있음.
    ic_color = 'white' # 아이콘 색상
    folium.Marker(
        location=[row['위도'],row['경도']],
        popup=row['주소'],
        tooltip=f"{row['상호']}({row['브랜드']})", # 팝업 예시 : xxx점(스타벅스)
        icon=folium.Icon(
            color=brand_colors.get(row['브랜드'],'gray'),
        icon_color='ic_color',
        icon='coffee',
        prefix='fa')
    ).add_to(filter_map)
#end for

htmlfile = dataOut + 'coffee_map02.html'
seoul_map.save(htmlfile)
print(f'{htmlfile} 파일 생성')

./../dataOut/coffee_map02.html 파일 생성


In [75]:
filter_map